<a href="https://www.kaggle.com/code/averma111/pytorch-timeseries-pss3e19?scriptVersionId=136491356" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [52]:
%%capture
!pip install pytorch_forecasting

In [53]:
import warnings
import numpy as np
import pandas as pd
import copy
from pathlib import Path
import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

In [54]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [55]:
train = pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv")
train.head()


,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               136950 non-null  int64  
 1   date             136950 non-null  object 
 2   country          136950 non-null  object 
 3   store            136950 non-null  object 
 4   product          136950 non-null  object 
 5   num_sold         136950 non-null  int64  
 6   sold_by_country  0 non-null       float64
dtypes: float64(1), int64(2), object(4)
memory usage: 7.3+ MB


In [87]:
product_by_sold=train.groupby(['product'])['num_sold'].sum().reset_index()
product_by_sold

,product,num_sold
0,Using LLMs to Improve Your Coding,6069281
1,Using LLMs to Train More LLMs,5993975
2,Using LLMs to Win Friends and Influence People,932564
3,Using LLMs to Win More Kaggle Competitions,5068313
4,Using LLMs to Write Better,4604192


In [97]:
fig = px.bar(product_by_sold,x='product',y='num_sold')
fig.show()

In [91]:
store_by_sold=train.groupby(['store'])['num_sold'].sum().reset_index()
store_by_sold

,store,num_sold
0,Kagglazon,15661335
1,Kaggle Learn,2755934
2,Kaggle Store,4251056


In [92]:
fig = px.bar(store_by_sold,x='store',y='num_sold')
fig.show()

In [93]:
country_by_sold=train.groupby(['country'])['num_sold'].sum().reset_index()
country_by_sold

,country,num_sold
0,Argentina,1683244
1,Canada,7013114
2,Estonia,3577319
3,Japan,6013786
4,Spain,4380862


In [94]:
fig = px.bar(country_by_sold,x='country',y='num_sold')
fig.show()

In [80]:
train.groupby(['date'])['num_sold'].sum()

date
2017-01-01    20086
2017-01-02    15563
2017-01-03    15039
2017-01-04    14516
2017-01-05    14083
              ...  
2021-12-27    16724
2021-12-28    18507
2021-12-29    20110
2021-12-30    20156
2021-12-31    20422
Name: num_sold, Length: 1826, dtype: int64